In [1]:
!pip install langgraph langchain_community bandit[toml] pandas -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.8/133.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.5 MB/s eta 0:00:00


In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict
import subprocess, json, pandas as pd, os, tempfile

In [3]:
repo_url = "https://github.com/sqlitebrowser/sqlitebrowser.git"
clone_dir = "/content/sqlitebrowser"
!git clone $repo_url $clone_dir
print("Repo cloned to", clone_dir)

Cloning into '/content/sqlitebrowser'...
remote: Enumerating objects: 27674, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 27674 (delta 2), reused 1 (delta 0), pack-reused 27663 (from 2)
Receiving objects: 100% (27674/27674), 34.26 MiB | 21.77 MiB/s, done.
Resolving deltas: 100% (19285/19285), done.
Repo cloned to /content/sqlitebrowser


In [4]:
def bandit_scan(path):
    cmd = ["bandit", "-r", path, "-f", "json"]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return json.loads(result.stdout) if result.stdout else {}

In [5]:
def commit_sentiment(repo_path):
    os.chdir(repo_path)
    msg = subprocess.check_output(["git", "log", "--oneline", "-1"], text=True)
    return "negative" if any(k in msg.lower() for k in ["fix","bug","error","vuln"]) else "neutral"

In [6]:
class State(TypedDict):
    repo_path: str
    bandit_out: dict
    sentiment: str
    issues: list

def scan_step(state):
    state["bandit_out"] = bandit_scan(state["repo_path"])
    state["issues"] = state["bandit_out"].get("results", [])
    return state

def sentiment_step(state):
    state["sentiment"] = commit_sentiment(state["repo_path"])
    return state

workflow = StateGraph(State)
workflow.add_node("scan", scan_step)
workflow.add_node("sentiment", sentiment_step)
workflow.add_edge("scan", "sentiment")
workflow.set_entry_point("scan")
workflow.add_edge("sentiment", END)

app = workflow.compile()
final_state = app.invoke({"repo_path": clone_dir})

In [7]:
df = pd.DataFrame(final_state["issues"])
df.to_csv("results.csv", index=False)
print("Saved", len(df), "rows to results.csv")
df.head()

Saved 3 rows to results.csv


,code,col_offset,end_col_offset,filename,issue_confidence,issue_cwe,issue_severity,issue_text,line_number,line_range,more_info,test_id,test_name
0,24 for i in range(rowcount):\n25 ...,29,64,/content/sqlitebrowser/tests/createtestdb.py,HIGH,"{'id': 330, 'link': 'https://cwe.mitre.org/dat...",LOW,Standard pseudo-random generators are not suit...,25,[25],https://bandit.readthedocs.io/en/1.8.6/blackli...,B311,blacklist
1,"25 text = """".join( [random.choice(...",18,33,/content/sqlitebrowser/tests/createtestdb.py,HIGH,"{'id': 330, 'link': 'https://cwe.mitre.org/dat...",LOW,Standard pseudo-random generators are not suit...,26,[26],https://bandit.readthedocs.io/en/1.8.6/blackli...,B311,blacklist
2,"25 text = """".join( [random.choice(...",36,59,/content/sqlitebrowser/tests/createtestdb.py,HIGH,"{'id': 330, 'link': 'https://cwe.mitre.org/dat...",LOW,Standard pseudo-random generators are not suit...,26,[26],https://bandit.readthedocs.io/en/1.8.6/blackli...,B311,blacklist
